## This notebook accompanies our submission for SWAT4HCLS 2023 "data-in-use", showcasing federated queries across UniProt, OMA and Bgee for identifying essential proteins

#### Note: we build the query progressively to illustrate step by step how to achieve the end result. We also highlight lessons learned for each of the steps, where relevant.

In [14]:
# helper functions
import sys
!{sys.executable} -m pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import sys, os, time
import pandas as pd

# always display full column results (don't truncate output)
pd.set_option('display.max_colwidth', -1)

OMA_SPARQL_endpoint = "https://sparql.omabrowser.org/sparql"
Bgee_SPARQL_endpoint = "https://bgee.org/sparql/"
UniProt_SPARQL_endpoint = "https://sparql.uniprot.org/sparql/"

# the endpoints must be defined as wrappers for executing SPARQL queries
sparql_endpoint_OMA = SPARQLWrapper(OMA_SPARQL_endpoint)
sparql_endpoint_UniProt = SPARQLWrapper(UniProt_SPARQL_endpoint)

# function to print in a table results of a SPARQL query
def pretty_print(results):
    
    # how to transform SPARQL results into Pandas dataframes
    
    # get header (column names) from results
    header = results["results"]["bindings"][0].keys()

    # display table of results:
    table = []
    
    # the SPARQL JSON results to the query are available in the "results", "bindings" entry:
    for entry in results["results"]["bindings"]:
        # append entries from the results to a regular Python list of rows, which we can then transform to a Pandas DF
        row = [entry[column]["value"] if entry.get(column, None) != None else None for column in header]
        table.append(row)
    df = pd.DataFrame(table, columns=list(header))
    return df



/var/folders/5d/3j6fyxq90gs2qdxgzpzsl1gh0000gp/T/ipykernel_54782/2522515982.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


### 1. Identify the taxon ID of "S. cerevisiae" in OMA RDF

#### First attempt: fetch directly from OMA

In [12]:
query_OMA_s_cerevisiae = """
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX obo: <http://purl.obolibrary.org/obo/>

select ?taxon ?name where {
    ?taxon  a up:Taxon.
    ?taxon up:scientificName ?name.
    filter (strstarts (lcase(?name), "saccharomyces cerevisiae" ))
}
"""

In [13]:
# set the query to be executed against the OMA endpoint and set the return format to JSON
sparql_endpoint_OMA.setQuery(query_OMA_s_cerevisiae)
sparql_endpoint_OMA.setReturnFormat(JSON)

results = sparql_endpoint_OMA.query().convert()

pretty_print(results)


,taxon,name
0,http://purl.uniprot.org/taxonomy/559292,Saccharomyces cerevisiae (strain ATCC 204508 / S288c)
1,http://purl.uniprot.org/taxonomy/764097,Saccharomyces cerevisiae (strain AWRI796)
2,http://purl.uniprot.org/taxonomy/764099,Saccharomyces cerevisiae (strain VIN 13)
3,http://purl.uniprot.org/taxonomy/764101,Saccharomyces cerevisiae (strain FostersO)


 Since there are multiple such taxa, we need to decide which one is the reference we should use in going forward. For this, since the OMA RDF does not store GO annotations, we will federate with UniProt to compute the number of annotations per taxon. We will consider the reference one to be the highest annotated one.

### 2. Combine with UniProt to identify taxon with most GO term annotations (this will be the reference one)
Note: despite how few taxa are transmitted from OMA to UniProt, this query takes very long to execute. This is  because all annotations need to be fetched from uniprot and sent back over before computing the aggregation.

In [18]:
%%time
query_OMA_UniProt_s_cerevisiae = """
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX obo: <http://purl.obolibrary.org/obo/>

select ?taxon (count (distinct ?goTerm) as ?num_annot) where {
    ?taxon  a up:Taxon.
    ?taxon up:scientificName ?name.
    filter (strstarts (lcase(?name), "saccharomyces cerevisiae" ))
    SERVICE <http://sparql.uniprot.org/sparql> {
        Select ?taxon ?goTerm where {
            ?protein a up:Protein .
            ?protein up:classifiedWith ?goTerm .
            ?protein  up:organism ?taxon .
        } 
    }
} group by ?taxon order by desc(?num_annot) # limit 1 -- if we only want to directly fetch the top one

"""

sparql_endpoint_OMA.setQuery(query_OMA_UniProt_s_cerevisiae)
sparql_endpoint_OMA.setReturnFormat(JSON)

results = sparql_endpoint_OMA.query().convert()

pretty_print(results)

CPU times: user 56.6 ms, sys: 14.9 ms, total: 71.5 ms
Wall time: 5min 37s


,taxon,num_annot
0,http://purl.uniprot.org/taxonomy/559292,6417
1,http://purl.uniprot.org/taxonomy/764097,81
2,http://purl.uniprot.org/taxonomy/764099,70
3,http://purl.uniprot.org/taxonomy/764101,60


### 2b. Execute same query at UniProt site (invoking OMA remotely)

In [17]:
%%time
query_UniProt_OMA = """
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX obo: <http://purl.obolibrary.org/obo/>

SELECT ?taxon (count (distinct ?goTerm) as ?num_annot) where {
  SERVICE<https://sparql.omabrowser.org/sparql>{
select ?taxon where {
?taxon  a up:Taxon.
?taxon up:scientificName ?name.
filter (strstarts (lcase(?name), "saccharomyces cerevisiae" ))
    } }
?protein a up:Protein .
    	?protein up:classifiedWith ?goTerm .
    	?protein  up:organism ?taxon . 
  
} group by ?taxon order by desc(?num_annot) # limit 1 -- if we only want to directly fetch the top one


"""

sparql_endpoint_UniProt.setQuery(query_UniProt_OMA)
sparql_endpoint_UniProt.setReturnFormat(JSON)

results = sparql_endpoint_UniProt.query().convert()

pretty_print(results)

CPU times: user 17.4 ms, sys: 5.18 ms, total: 22.6 ms
Wall time: 1.28 s


,num_annot,taxon
0,6417,http://purl.uniprot.org/taxonomy/559292
1,81,http://purl.uniprot.org/taxonomy/764097
2,70,http://purl.uniprot.org/taxonomy/764099
3,60,http://purl.uniprot.org/taxonomy/764101


The previous query illustrates that the taxon we are looking for is certainly NCBI ID 559292. We will use this throughout the remainder of the tutorial directly.

###### **LESSON LEARNED no. 1: generating and transfering large intermediate results can make order of magnitude difference in performance. However, it is not always easy to predict which query fragments will lead to this problem.**


### 3. Compute orthologs of S. cerevisiae in reference species. Here, we will restrict refences species to be those available in Bgee, as those are the only ones for which we can further compute expression profiles

In [ ]:
.....

### 4. Having computed orthologs, now also compute where these are highly expressed (federated query OMA + BGEE)

In [ ]:
....

### 5. Attempt aggregation via SPARQL: compute number of orthologs and average expression breadth per ortholog

In [ ]:
....

### 5b. Variant: compute aggregation directly in Python

In [ ]:
....

### 6. Putting everything together: the query that should unite all information in this tutorial (note: will not run due to timeouts)

In [ ]:
.....

### Caveat: expression as hierarchy - are the same things counted twice? How do we take only the leaves? (use UBERON)